In [ ]:
import pandas as pd
import numpy as np
import gensim
from gensim.models import Word2Vec
from gensim.models import FastText
from sklearn.decomposition import PCA
from matplotlib import pyplot

In [ ]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import string
import re
import nltk
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
df_brand = pd.read_csv('/content/brand_category.csv')
df_brand.head()

,BRAND,BRAND_BELONGS_TO_CATEGORY,RECEIPTS
0,CASEYS GEN STORE,Tobacco Products,2950931
1,CASEYS GEN STORE,Mature,2859240
2,EQUATE,Hair Removal,893268
3,PALMOLIVE,Bath & Body,542562
4,DAWN,Bath & Body,301844


In [ ]:
df_brand.shape

(9906, 3)

In [ ]:
df_category = pd.read_csv('/content/categories.csv')
df_category.head()

,CATEGORY_ID,PRODUCT_CATEGORY,IS_CHILD_CATEGORY_TO
0,1f7d2fa7-a1d7-4969-aaf4-1244f232c175,Red Pasta Sauce,Pasta Sauce
1,3e48a9b3-1ab2-4f2d-867d-4a30828afeab,Alfredo & White Pasta Sauce,Pasta Sauce
2,09f3decc-aa93-460d-936c-0ddf06b055a3,Cooking & Baking,Pantry
3,12a89b18-4c01-4048-94b2-0705e0a45f6b,Packaged Seafood,Pantry
4,2caa015a-ca32-4456-a086-621446238783,Feminine Hygeine,Health & Wellness


In [ ]:
df_category.shape

(118, 3)

In [ ]:
df_retailer = pd.read_csv('/content/offer_retailer.csv')
df_retailer.head()

,OFFER,RETAILER,BRAND
0,Spend $50 on a Full-Priced new Club Membership,SAMS CLUB,SAMS CLUB
1,"Beyond Meat® Plant-Based products, spend $25",NaN,BEYOND MEAT
2,Good Humor Viennetta Frozen Vanilla Cake,NaN,GOOD HUMOR
3,"Butterball, select varieties, spend $10 at Dil...",DILLONS FOOD STORE,BUTTERBALL
4,"GATORADE® Fast Twitch®, 12-ounce 12 pack, at A...",AMAZON,GATORADE


In [ ]:
df_retailer.shape

(384, 3)

In [ ]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity



brand_df = pd.read_csv('/content/brand_category.csv')
category_df = pd.read_csv('/content/categories.csv')
retailer_df = pd.read_csv('/content/offer_retailer.csv')

# Preprocess data
def preprocess_text(text):
    return str(text).lower().split()

brand_df['BRAND'] = brand_df['BRAND'].apply(preprocess_text)
brand_df['BRAND_BELONGS_TO_CATEGORY'] = brand_df['BRAND_BELONGS_TO_CATEGORY'].apply(preprocess_text)
category_df['PRODUCT_CATEGORY'] = category_df['PRODUCT_CATEGORY'].apply(preprocess_text)
retailer_df['OFFER'] = retailer_df['OFFER'].apply(preprocess_text)

# Train a Word2Vec model
all_sentences = pd.concat([
    brand_df['BRAND'],
    brand_df['BRAND_BELONGS_TO_CATEGORY'],
    category_df['PRODUCT_CATEGORY'],
    retailer_df['OFFER']
], ignore_index=True)

model = Word2Vec(sentences=all_sentences, vector_size=100, window=5, min_count=1, workers=4)

# Function to compute vector representation of a text
def get_vector(text):
    vector = np.zeros(100)
    count = 0
    for word in text:
        if word in model.wv:
            vector += model.wv[word]
            count += 1
    if count:
        vector /= count
    return vector

# Get vectors for offers
retailer_df['OFFER_VECTOR'] = retailer_df['OFFER'].apply(get_vector)

# Function to get relevant offers
def get_relevant_offers(query):
    query_vector = get_vector(preprocess_text(query))
    similarities = [cosine_similarity([query_vector], [offer_vector])[0][0] for offer_vector in retailer_df['OFFER_VECTOR']]
    retailer_df['SIMILARITY'] = similarities
    top_offers = retailer_df.sort_values(by='SIMILARITY', ascending=False).head(3)  # top 3 offers
    return top_offers[['OFFER', 'SIMILARITY']]

print(get_relevant_offers("amazon"))
#print(get_relevant_offers("meat products"))
#print(get_relevant_offers("spend $25"))


                                                 OFFER  SIMILARITY
255  [starry™, lemon, lime, soda,, 7.5-ounce, 10, p...    0.994370
341  [perfect, keto, super, reds,, online, at, amazon]    0.993931
4    [gatorade®, fast, twitch®,, 12-ounce, 12, pack...    0.993915
